In [0]:
%sql
drop catalog demo1 cascade

####Copy Table as a bulk from ADLS GEN2

In [0]:
# Base ADLS Gen2 path (container level)
source_base_path = "abfss://target@adventurestorage2016.dfs.core.windows.net/"

# External storage location for Delta tables (must be accessible & registered in UC if needed)
external_base_path = "abfss://bronze@adventurestorage2016.dfs.core.windows.net/"

# UC Catalog
catalog_name = "adventure_works2016"

# List all schema folders inside the container
schemas_df = dbutils.fs.ls(source_base_path)

for schema_folder in schemas_df:
    schema_name = schema_folder.name.rstrip("/")
    schema_name_uc = schema_name.lower()

    # Create schema if it doesn't exist
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name_uc}")

    # List all tables inside the schema folder
    tables_df = dbutils.fs.ls(schema_folder.path)
    for table_folder in tables_df:
        table_name = f"bronze_{table_folder.name.rstrip('/').lower()}"
        source_path = table_folder.path

        # External location for Delta data
        target_path = f"{external_base_path}{schema_name_uc}/bronze_{table_name}"

        print(f"Processing: {source_path} → {catalog_name}.{schema_name_uc}.{table_name}")
        print(f"External location: {target_path}")

        # Read parquet
        df = spark.read.parquet(source_path)

        # Write Delta data to external location
        df.write.format("delta").mode("overwrite").save(target_path)

        # Register external table in Unity Catalog
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name_uc}.{table_name}
            USING DELTA
            LOCATION '{target_path}'
        """)

        print(f"✅ Loaded into {catalog_name}.{schema_name_uc}.{table_name} (External at {target_path})")


In [0]:
%sql
Select * from adventure_works2016.production.bronze_productcosthistory